# Version control the parameter files

Here we learn a nice little trick to version-control the parameter files while tracking the simulation output directories for all the versions of the network.

In [1]:
import denest
import nest
import yaml
from pathlib import Path
from pprint import pprint

In [2]:
PARAMS_DIR = Path('./data/params')  # Where all the parameter files are
DATA_DIR = Path('./data/outputs')  # Where all the simulation output directories are

## 1. Turn ``PARAMS_DIR`` into a git directory

## 2. Make modifications to the parameters 

## 3. Run a simulation with the modified parameter files and track the output directory

### A. Generate a unique output directory name

For example, using the current date and time:

In [3]:
from datetime import datetime

output_dir_name = datetime.now().strftime("%Y-%m-%d_%H:%M:%S")

output_dir = Path(DATA_DIR)/output_dir_name
output_dir

PosixPath('data/outputs/2020-06-28_20:34:59')

### B. Create a symlink to the output directory in the version controlled `PARAMS_DIR`

In [4]:
def overwrite_dir_symlink(target, link_path):
    """Overwrite or create a symlink to a directory.
    
    ``target`` should be relative to ``link_path``"""
    import os, errno
    
    try:
        os.symlink(target, link_path)
    except OSError as e:
        if e.errno == errno.EEXIST:
            os.remove(link_path)
            os.symlink(target, link_path)
        else:
            raise e

In [5]:
# Needs path RELATIVE to symlink position 
overwrite_dir_symlink(Path('../outputs')/output_dir_name, PARAMS_DIR/'output')

### C. Run the simulation

In [6]:
denest.run(PARAMS_DIR/'tree_paths.yml', output_dir=output_dir)

2020-06-28 20:34:59,822 [denest] INFO: 

=== RUNNING SIMULATION ========================================================

2020-06-28 20:34:59,823 [denest] INFO: Loading parameter file paths from data/params/tree_paths.yml
2020-06-28 20:34:59,825 [denest] INFO: Finished loading parameter file paths
2020-06-28 20:34:59,828 [denest] INFO: Loading parameters files: 
['./network_tree.yml',
 './simulation.yml',
 './session_models.yml',
 './kernel.yml']
2020-06-28 20:34:59,882 [denest] INFO: Initializing simulation...
2020-06-28 20:34:59,930 [denest.utils.validation] INFO: Object `simulation`: params: using default value for optional parameters:
{'input_dir': 'input'}
2020-06-28 20:34:59,934 [denest.simulation] INFO: Initializing NEST kernel and seeds...
2020-06-28 20:34:59,935 [denest.simulation] INFO:   Resetting NEST kernel...
2020-06-28 20:34:59,970 [denest.simulation] INFO:   Setting NEST kernel status...
2020-06-28 20:34:59,989 [denest.simulation] INFO:     Calling `nest.SetKernelStatus

The simulation output was saved in the unique output directory created.  

In [7]:
print(output_dir)
!ls {output_dir}

data/outputs/2020-06-28_20:34:59
data               git_hash           parameter_tree.yml session_times.yml


The `PARAMS_DIR/output` symlink points to the actual output directory.

In [8]:
!ls {PARAMS_DIR/'output/'}

data               git_hash           parameter_tree.yml session_times.yml


## 4. Commit the `PARAMS_DIR` with the updated params and `output` symlimk